# Análise comparativa

In [1]:
from IPython.display import display, Markdown
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

## 1. Obtenção dos dados:

In [2]:
df = pd.read_csv("../data/raw/day.csv")
df['dteday'] = pd.to_datetime(df['dteday'])
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,instant,Índice de registro,quantitativa,discreta
1,dteday,Data,qualitativa,ordinal
2,season,Estação do ano,qualitativa,ordinal
3,yr,Ano,qualitativa,ordinal
4,mnth,Mês,qualitativa,ordinal
5,holiday,Se o dia é feriado ou não,qualitativa,nominal
6,weekday,Dia da semana,qualitativa,nominal
7,workingday,Se o dia não é fim de semana e nem feriado,qualitativa,nominal
8,weathersit,Clima,qualitativa,nominal
9,temp,Temperatura normalizada em Celsius,quantitativa,contínua


---

## 2. Preparação de dados:

In [3]:
print("Dados faltantes por coluna:")
print(df.isnull().sum())

Dados faltantes por coluna:
instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64


In [4]:
target_column = 'cnt'

nominal_columns = (
    df_dict
    .query("subtipo == 'nominal'")
    .variavel
    .to_list()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'contínua'")
    .variavel
    .to_list()
)

ordinal_columns = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != 'dteday'")
    .variavel
    .to_list()
)

discrete_columns = (
    df_dict
    .query("subtipo == 'discreta' and variavel != @target_column")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [5]:
if 'dteday' in X.columns:
    X['year'] = X['dteday'].dt.year
    X['month'] = X['dteday'].dt.month
    X['day'] = X['dteday'].dt.day
    X = X.drop(columns=['dteday'])

nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), 
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), 
    ('normalization', StandardScaler())  
])

continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  
    ('normalization', StandardScaler()) 
])

ordinal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),
    ('encoding', OrdinalEncoder()), 
    ('normalization', StandardScaler())
])

discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), 
    ('normalization', StandardScaler()) 
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('ordinal', ordinal_preprocessor, ordinal_columns),
    ('discrete', discrete_preprocessor, discrete_columns),
    ('continuous', continuous_preprocessor, continuous_columns)
])

X_preprocessed = continuous_preprocessor.fit_transform(df[continuous_columns])
model = LinearRegression()

final_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)
])

In [6]:
X_prepared = preprocessor.fit_transform(X)
X_prepared.shape

(731, 20)

In [7]:
X_prepared = preprocessor.fit_transform(X)
print("Dados pré-processados:")
print(X_prepared)

Dados pré-processados:
[[-0.171981   -0.40955052 -0.40727045 ... -0.67994602  1.25017133
  -0.38789169]
 [-0.171981   -0.40955052 -0.40727045 ... -0.74065231  0.47911298
   0.74960172]
 [-0.171981    2.4417012  -0.40727045 ... -1.749767   -1.33927398
   0.74663186]
 ...
 [-0.171981   -0.40955052 -0.40727045 ... -1.42434419  0.87839173
  -0.85355213]
 [-0.171981   -0.40955052 -0.40727045 ... -1.49004895 -1.01566357
   2.06944426]
 [-0.171981    2.4417012  -0.40727045 ... -1.54048197 -0.35406086
  -0.46020122]]


---

## 3. Seleção de modelos


Iremos análisar quatro modelos, que serão testados utilizando um método de validação, a saber:

- K-Nearest Neighbors
- Gradient Boosting
- Decision Tree
- Random Forest

Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Utilizaremos as seguintes métricas para análise:

- **Neg Mean Squared Error ('neg_mean_squared_error')**: O erro quadrático médio (MSE) é a média dos quadrados das diferenças entre os valores previstos e os valores reais. É uma medida que penaliza erros maiores de forma mais severa, já que os erros são elevados ao quadrado.
- **Neg Mean Absolute Error ('neg_mean_absolute_error')**: O erro absoluto médio (MAE) é a média das diferenças absolutas entre os valores previstos e os valores reais. Ao contrário do MSE, o MAE não penaliza erros maiores mais severamente.
- **R² ('r2')**: O R², ou coeficiente de determinação, mede a proporção da variabilidade total dos dados que é explicada pelo modelo. Em outras palavras, indica o quão bem os valores previstos se ajustam aos valores reais.

In [8]:
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = .2
random_state = 42
scoring = 'neg_mean_squared_error'
metrics = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']

max_iter = 1000 
models = [
    ('K-Nearest Neighbors', KNeighborsRegressor(), {
        "n_neighbors": range(3, 20, 2), 
        'weights': ['uniform', 'distance']
    }),
    ('Gradient Boosting', GradientBoostingRegressor(random_state=random_state), {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }),
     ('Decision Tree',  DecisionTreeRegressor(random_state=random_state), {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'], 
        'max_depth': [3, 6, 8]
    }),
    ('Random Forest',  RandomForestRegressor(random_state=random_state), {
        'criterion': ['squared_error', 'absolute_error'],
        'max_depth': [3, 6, 8], 
        'n_estimators': [10, 30]
    }),
]

In [9]:
results = pd.DataFrame({})
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)

for model_name, model_object, model_parameters in models:
    print(f"running {model_name}...")
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )
    
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    
    scores = cross_validate(
        estimator=approach,
        X=X,
        y=y,
        cv=cross_validate_comparative_analysis,
        n_jobs=-1,
        scoring=metrics
    )
    
    scores_df = pd.DataFrame(scores)
    aggregated_scores = scores_df.agg(['mean', 'std'])
    aggregated_scores['model_name'] = model_name
    display(aggregated_scores)
    results = pd.concat([results, aggregated_scores], ignore_index=True)

running K-Nearest Neighbors...


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_r2,model_name
mean,0.778052,0.029866,-495085.285587,-521.421224,0.867283,K-Nearest Neighbors
std,0.125487,0.014388,98807.812136,56.917455,0.024541,K-Nearest Neighbors


running Gradient Boosting...


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_r2,model_name
mean,42.909599,0.016341,-11775.561442,-68.763639,0.996826,Gradient Boosting
std,8.347175,0.001307,3159.517240,7.482689,0.000856,Gradient Boosting


running Decision Tree...


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_r2,model_name
mean,0.684064,0.018473,-59390.980641,-164.245985,0.983967,Decision Tree
std,0.106449,0.006129,12130.331634,17.579599,0.003618,Decision Tree


running Random Forest...


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_r2,model_name
mean,10.112643,0.025550,-18307.274898,-83.639403,0.995059,Random Forest
std,1.794122,0.014259,5609.330859,11.129464,0.001523,Random Forest


In [10]:
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

display(Markdown("### 3.1 Resultados gerais"))
(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

### 3.1 Resultados gerais

- Gradient Boosting foi o modelo mais eficaz no geral, com o menor erro e maior coeficiente de determinação, embora tenha um custo computacional mais elevado.
- Random Forest também teve um bom desempenho, mas com um tempo de execução menor que o Gradient Boosting, o que pode ser um trade-off interessante.
- Decision Tree foi o mais rápido, mas sacrificou precisão em relação aos outros modelos.
- KNN teve um desempenho inferior em todos os aspectos, sugerindo que pode não ser a melhor escolha para este problema específico.

Esses resultados sugerem que, se o objetivo é maximizar a precisão e há tempo computacional disponível, o Gradient Boosting é a melhor opção. Se o tempo de treinamento for uma restrição, o Random Forest pode ser uma boa alternativa.

## 3.2 Persistência do modelo

In [11]:
model_name, model_object, model_parameters  = [foo for foo in models if foo[0] == "Gradient Boosting"][0] 

model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )

approach = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model_grid_search)
])

approach.fit(X, y)

print(f"Hiper parâmetros do modelo: {approach.steps[1][1].best_params_}")

Hiper parâmetros do modelo: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 150}


In [12]:
joblib.dump(approach, '../models/model.joblib')

['../models/model.joblib']